In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import shutil
import re
import time
import json
import unicodedata
import html
import re
from functools import reduce

In [2]:
def clean_text(text):
    text = text.replace("\r\n", " ")
    text = text.replace("\n", " ")
    text = text.replace("\r", " ")
    text = text.replace("\t", " ")
    text = text.replace(u'\xa0', u' ')
    text.strip()
    return text



In [3]:
main_url = "http://www.gardening.cornell.edu/homegardening/sceneb771.html"
base_url = "http://www.gardening.cornell.edu/homegardening/"
page = requests.get(main_url)
soup = BeautifulSoup(page.content, 'html.parser')
html_pages = []
for link in soup.find_all('a'):
    html_pages.append(link.get('href'))
formatted_pages = []
for page in html_pages:
    if page != None and "scene" in str(page):
        formatted = base_url + page
        formatted_pages.append(formatted)

In [21]:
final_dict = {}
for url in formatted_pages:
    
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')

    bolds = soup.find_all(['b'])
    name = bolds[1].text.strip()
    
    moistures = soup.find_all(['li'])
    moist_content = [m.contents for m in moistures]
    moist_content = list(map(lambda l : l[0], moist_content))
    moist_content = list(filter(lambda m : "soil" in m, moist_content))
    
    data_dict = {}
    headers = soup.find_all(['p', 'li', 'b'])
    total_text = []
    for head in headers:
        total_text.append(head.text)

    total_text = list(map(lambda t : clean_text(t), total_text))
    occurs1 = occurs2 = occurs3 = occurs4 = occurs5 = occurs6 = occurs7 = occurs8 = occurs9 = True
    get_next = False
    for text in total_text:
        if "Germination temperature:" in text and occurs1:
            text = text.replace("Germination temperature:", "").strip()
            data_dict["Germ_Temp"] = text
            occurs1 = False
        if "Days to emergence:" in text and occurs2:
            data_dict["Germ_Time"] = text.strip()
            occurs2 = False
        if "Height:" in text and occurs3:
            text = text.replace("Height:", "").strip()
            data_dict["Height"] = text
            occurs3 = False
        if "Spread:" in text and occurs4:
            text = text.replace("Spread:", "").strip()
            data_dict["Spread"] = text
            occurs4 = False
        if "Lifecycle:" in text and occurs5:
            text = text.replace("Lifecycle:", "").strip()
            data_dict["Lifecycle"] = text
            occurs5 = False
        if "Ease-of-care:" in text and occurs6:
            text = text.replace("Ease-of-care:", "").strip()
            data_dict["Care"] = text
            occurs6 = False
        if "sun" in text and occurs7:
            data_dict["Sun"] = text
            occurs7 = False
        
#         if "Soil conditions" in text and occurs8 and not get_next:
#             occurs8 = False
#             get_next = True
#         if not occurs8 and get_next:
#             text = text.strip()
#             data_dict["Soil"] = text
#             get_next = False
        if "shade" in text and occurs9:
            data_dict["shade"] = text
            occurs9 = False
    
    data_dict["Soil_Moisture"] = moist_content

    combined_text = reduce(lambda s, acc : acc + s, total_text)
    combined_text = combined_text.replace("\n", ";;")
    data_dict["Combined"] = combined_text
    
    final_dict[name] = data_dict
    
for plant in final_dict:
    inner_dict = final_dict[plant]
    if inner_dict["Lifecycle"] == "annual":
        inner_dict["Soil Depth"] = "3 Inches"
    elif inner_dict["Lifecycle"] == "perennial":
        inner_dict["Soil Depth"] = "Up to 24 Inches"

# scrape off moisture by looking at moisture <b> tag, and if tag says Soil conditions, record after anything iwht soil in at, at most 5 bullets with <li?>

{'Tomatoes': {'Sun': 'full sun', 'Lifecycle': 'annual', 'Care': 'moderately difficult', 'Height': '2 to 6 feet', 'Spread': '2 to 6 feet', 'Germ_Temp': '60 F to 95 F       - Germinates best at 75 F to 90 F.  Germinates very slowly at cooler temperatures.', 'Germ_Time': 'Days to emergence: 6 to 12       - About 1 week at 75 F.', 'shade': 'Do not remove leaves that shade fruit to prevent sunscald.  ', 'Soil_Moisture': ['tolerates acid soil', 'requires well-drained soil', 'Keep\r\nsoil evenly moist to prevent blossom end rot. This can also help\r\nprevent cracking when fruit absorbs water too fast after heavy rain\r\nfollowing dry conditions. ', 'Catfacing\r\n(misshapen, deformed fruit) is caused by incomplete pollination,\r\nusually due to cold weather. Don’t rush to transplant until weather has\r\nstabilized and soil is warm. '], 'Combined': "©2006 Cornell University. All rights reserved.Big RainbowBlack from TulaBlack PrinceBox Car WillieBrandywineCherokee PurpleGolden QueenGreen ZebraM

In [80]:
fp = open("initial_plant_scrape.json", "w")
json.dump(final_dict, fp)
fp.close()

In [4]:
# Add in the soil depth
# based on https://theplantium.com/2017/09/28/soil-depth-and-plant-selection/

fp = open("initial_plant_scrape.json", "r")
final_dict = json.load(fp)
fp.close()


for plant in final_dict:
    inner_dict = final_dict[plant]
    if inner_dict["Lifecycle"] == "annual":
        inner_dict["Soil Depth"] = "3 Inches"
    elif inner_dict["Lifecycle"] == "perennial":
        inner_dict["Soil Depth"] = "Up to 24 Inches"

fp = open("initial_plant_scrape.json", "w")
json.dump(final_dict, fp)
fp.close()

{'Tomatoes': {'Sun': 'full sun', 'Lifecycle': 'annual', 'Care': 'moderately difficult', 'Height': '2 to 6 feet', 'Spread': '2 to 6 feet', 'Germ_Temp': '60 F to 95 F       - Germinates best at 75 F to 90 F.  Germinates very slowly at cooler temperatures.', 'Germ_Time': 'Days to emergence: 6 to 12       - About 1 week at 75 F.', 'Combined': "©2006 Cornell University. All rights reserved.Big RainbowBlack from TulaBlack PrinceBox Car WillieBrandywineCherokee PurpleGolden QueenGreen ZebraMoskvichMr. StripeyStriped GermanStriped RomanTappy's Heritage     Heirlooms:ClassicaLa RomaLa RosaNovaPlum DandyRomaViva Italia Paste:                Basket VeeBetter BoyBig BeefBig BoyCelebrityJet StarMountain FreshMountain SpringSunbeamSunriseSupersonicUltra SweetMain Season:CascadeEarly GirlGold DustLemon BoySunriseTaxi Early:Cosmonaut VolkoveCurrant, DaybreakEarly CascadeGold Rush Extra Early:JubileeEarly CherryFruity OrangeSarah's Goldstar CherrySungoldSupersweet 100  Grape:                Cherry:Some

TypeError: string indices must be integers

In [29]:
# clean the data for future use
fp = open("initial_plant_scrape.json", "r")
final_dict = json.load(fp)
fp.close()

In [30]:
schema = {"Height" : "Feet", "Spread" : "Feet", "Germ_Temp" : "Fahrenheit", "Germ_Time" : "Days"}

In [31]:
new_dict = {}
new_dict["schema"] = schema
new_dict["data"] = {}

In [32]:
numeric_regex = re.compile(r'[0-9][0-9]*')

def clean(string, regex=numeric_regex):
    matches = re.findall(regex, string)
    if len(matches) > 2:
        matches = matches[:2]
    return sorted(matches)

In [37]:
for key in final_dict:
    new_dict["data"][key] = final_dict[key]
    if 'Height' in final_dict[key]:
        new_dict["data"][key]["Cleaned_Height"] = clean(final_dict[key]['Height'])
    if 'Spread' in final_dict[key]:
        new_dict["data"][key]["Cleaned_Spread"] = clean(final_dict[key]['Spread'])
    if 'Germ_Temp' in final_dict[key]:
        new_dict["data"][key]["Cleaned_Germ_Temp"] = clean(final_dict[key]['Germ_Temp'])
    if 'Germ_Time' in final_dict[key]:
        new_dict["data"][key]["Cleaned_Germ_Time"] = clean(final_dict[key]['Germ_Time'])

In [ ]:
fp = open("cleaned_plant_scrape.json", "w")
json.dump(new_dict, fp)
fp.close()

In [ ]:
# Newest version of scrape as of July 1st 2020
import requests
from bs4 import BeautifulSoup
import pandas as pd
import shutil
import re
import time
import json
import unicodedata
import html
import re
from functools import reduce

# utilities
numeric_regex = re.compile(r'[0-9][0-9]*')

def clean(string, regex=numeric_regex):
    matches = re.findall(regex, string)
    if len(matches) > 2:
        matches = matches[:2]
    return sorted(matches)

def clean_text(text):
    text = text.replace("\r\n", " ")
    text = text.replace("\n", " ")
    text = text.replace("\r", " ")
    text = text.replace("\t", " ")
    text = text.replace(u'\xa0', u' ')
    text.strip()
    return text

# get pages
main_url = "http://www.gardening.cornell.edu/homegardening/sceneb771.html"
base_url = "http://www.gardening.cornell.edu/homegardening/"
page = requests.get(main_url)
soup = BeautifulSoup(page.content, 'html.parser')
html_pages = []
for link in soup.find_all('a'):
    html_pages.append(link.get('href'))
formatted_pages = []
for page in html_pages:
    if page != None and "scene" in str(page):
        formatted = base_url + page
        formatted_pages.append(formatted)

# pull data off pages
final_dict = {}
for url in formatted_pages:
    
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')

    bolds = soup.find_all(['b'])
    name = bolds[1].text.strip()
    
    moistures = soup.find_all(['li'])
    moist_content = [m.contents for m in moistures]
    moist_content = list(map(lambda l : l[0], moist_content))
    moist_content = list(filter(lambda m : "soil" in m, moist_content))
    
    data_dict = {}
    headers = soup.find_all(['p', 'li', 'b'])
    total_text = []
    for head in headers:
        total_text.append(head.text)

    total_text = list(map(lambda t : clean_text(t), total_text))
    occurs1 = occurs2 = occurs3 = occurs4 = occurs5 = occurs6 = occurs7 = occurs8 = occurs9 = True
    get_next = False
    for text in total_text:
        if "Germination temperature:" in text and occurs1:
            text = text.replace("Germination temperature:", "").strip()
            data_dict["Germ_Temp"] = text
            occurs1 = False
        if "Days to emergence:" in text and occurs2:
            data_dict["Germ_Time"] = text.strip()
            occurs2 = False
        if "Height:" in text and occurs3:
            text = text.replace("Height:", "").strip()
            data_dict["Height"] = text
            occurs3 = False
        if "Spread:" in text and occurs4:
            text = text.replace("Spread:", "").strip()
            data_dict["Spread"] = text
            occurs4 = False
        if "Lifecycle:" in text and occurs5:
            text = text.replace("Lifecycle:", "").strip()
            data_dict["Lifecycle"] = text
            occurs5 = False
        if "Ease-of-care:" in text and occurs6:
            text = text.replace("Ease-of-care:", "").strip()
            data_dict["Care"] = text
            occurs6 = False
        if "sun" in text and occurs7:
            data_dict["Sun"] = text
            occurs7 = False
        if "shade" in text and occurs9:
            data_dict["shade"] = text
            occurs9 = False
    
    data_dict["Soil_Moisture"] = moist_content

    combined_text = reduce(lambda s, acc : acc + s, total_text)
    combined_text = combined_text.replace("\n", ";;")
    data_dict["Combined"] = combined_text
    
    final_dict[name] = data_dict

# add in plant soil depths
for plant in final_dict:
    inner_dict = final_dict[plant]
    if inner_dict["Lifecycle"] == "annual":
        inner_dict["Soil Depth"] = "3 Inches"
    elif inner_dict["Lifecycle"] == "perennial":
        inner_dict["Soil Depth"] = "Up to 24 Inches"

# make final dictionary
schema = {"Height" : "Feet", "Spread" : "Feet", "Germ_Temp" : "Fahrenheit", "Germ_Time" : "Days"}

new_dict = {}
new_dict["schema"] = schema
new_dict["data"] = {}

for key in final_dict:
    new_dict["data"][key] = final_dict[key]
    if 'Height' in final_dict[key]:
        new_dict["data"][key]["Cleaned_Height"] = clean(final_dict[key]['Height'])
    if 'Spread' in final_dict[key]:
        new_dict["data"][key]["Cleaned_Spread"] = clean(final_dict[key]['Spread'])
    if 'Germ_Temp' in final_dict[key]:
        new_dict["data"][key]["Cleaned_Germ_Temp"] = clean(final_dict[key]['Germ_Temp'])
    if 'Germ_Time' in final_dict[key]:
        new_dict["data"][key]["Cleaned_Germ_Time"] = clean(final_dict[key]['Germ_Time'])

fp = open("with_soil_moisture_plant_scrape.json", "w")
json.dump(new_dict, fp)
fp.close()


In [2]:
# clean data for website on 8/11/2020
import requests
from bs4 import BeautifulSoup
import shutil
import re
import time
import json
import unicodedata
import html
import re

In [3]:
# clean the data for website usage in json format for HTML
fp = open("cleaned_plant_scrape.json", "r")
cleaned_dict = json.load(fp)
fp.close()

In [7]:
data = cleaned_dict["data"]

html_dict = {}
for key in data:
    plant_dict = data[key]
    subdict = {}
    if "Sun" in plant_dict:
        subdict["Sunlight Conditions"] = plant_dict["Sun"]
    if "Lifecycle" in plant_dict:
        subdict["Plant Lifecycle"] = plant_dict["Lifecycle"]
    if "Care" in plant_dict:
        subdict["Plant Care Difficulty"] = plant_dict["Care"]
    if "Height" in plant_dict:
        subdict["Height"] = plant_dict["Height"]
    if "Spread" in plant_dict:
        subdict["Spread"] = plant_dict["Spread"]
    if "Germ_Temp" in plant_dict:
        subdict["Optimal Germination Temperature"] = plant_dict["Germ_Temp"]
    if "Germ_Time" in plant_dict:
        subdict["Time to Germination"] = plant_dict["Germ_Time"]
    if "Soil Depth" in plant_dict:
        subdict["Soil Depth"] = plant_dict["Soil Depth"]
    if "Combined" in plant_dict:
        subdict["Full Summary"] = plant_dict["Combined"]
    html_dict[key] = subdict
    
fp = open("ui_plant_scrape.json", "w")
json.dump(html_dict, fp)
fp.close()

In [8]:
# add in images to html ui
fp = open("ui_plant_scrape.json", "r")
cleaned_dict = json.load(fp)
fp.close()

plant_names = [name for name in cleaned_dict]

In [9]:
url_list = [
    "https://media.istockphoto.com/photos/tomato-isolated-on-white-background-picture-id466175630?k=6&m=466175630&s=612x612&w=0&h=fu_mQBjGJZIliOWwCR0Vf2myRvKWyQDsymxEIi8tZ38=",
    "https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Fupload.wikimedia.org%2Fwikipedia%2Fcommons%2Fthumb%2F1%2F1f%2FPoivrons_Luc_Viatour.jpg%2F1200px-Poivrons_Luc_Viatour.jpg&f=1&nofb=1",
    "https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Ftse1.mm.bing.net%2Fth%3Fid%3DOIP.w4zUpDHA0MdbFXsliwkI8gHaFL%26pid%3DApi&f=1",
    "https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Ftse1.mm.bing.net%2Fth%3Fid%3DOIP.O8avSAUC2FPsSiNGJNJdZgHaHa%26pid%3DApi&f=1",
    "https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Ftse1.mm.bing.net%2Fth%3Fid%3DOIP.mgVpq7ciCbCyL3wX6CZT8AHaD2%26pid%3DApi%26h%3D160&f=1",
    "https://duckduckgo.com/i/ec50a413.jpg",
    "https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Ftse1.mm.bing.net%2Fth%3Fid%3DOIP.TcWSa-oI-1tFcAxVNLYiMwHaEK%26pid%3DApi&f=1",
    "https://duckduckgo.com/i/b1e10561.jpg",
    "https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Ftse1.mm.bing.net%2Fth%3Fid%3DOIP.Aomm1eAzl9kYeX0lIHp7MwHaHa%26pid%3DApi%26h%3D160&f=1",
    "https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Ftse1.mm.bing.net%2Fth%3Fid%3DOIP.NK6QEoYO4p5cRInNPO4mbwHaHa%26pid%3DApi%26h%3D160&f=1",
    "https://duckduckgo.com/i/361b924d.png",
    "https://duckduckgo.com/i/2479d8e5.jpg",
    "https://duckduckgo.com/i/b3c94767.jpg",
    "https://duckduckgo.com/i/2f5ee178.jpg",
    "https://duckduckgo.com/i/617d8e7d.jpg",
    "https://duckduckgo.com/i/b4c78134.jpg",
    "https://duckduckgo.com/i/150de7e1.jpg",
    "https://duckduckgo.com/i/bf894751.jpg",
    "https://duckduckgo.com/i/708921e4.jpg",
    "https://duckduckgo.com/i/c6567859.jpg",
    "https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Ftse1.mm.bing.net%2Fth%3Fid%3DOIP.1ILchOtpVThWQ8a9jH5ZSQHaD4%26pid%3DApi&f=1",
    "https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Ftse1.mm.bing.net%2Fth%3Fid%3DOIP.YNrHZx0yfwwZaF0rJGFitwHaEe%26pid%3DApi&f=1",
    "https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Ftse1.mm.bing.net%2Fth%3Fid%3DOIP.h3-UYrAiLBzj62zZ6IfGBAHaFj%26pid%3DApi&f=1",
    "https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Ftse1.mm.bing.net%2Fth%3Fid%3DOIP.wkoBZ916eQWsZYIKBrrPMgHaE8%26pid%3DApi&f=1",
    "https://duckduckgo.com/i/1e52cf69.jpg",
    "https://duckduckgo.com/i/dfc793f9.jpg",
    "https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Ftse1.mm.bing.net%2Fth%3Fid%3DOIP.ejObMlnt9X9bGbOy7Lid9AHaE7%26pid%3DApi%26h%3D160&f=1",
    "https://duckduckgo.com/i/6d079b67.jpg",
    "https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Ftse1.mm.bing.net%2Fth%3Fid%3DOIP.Bp1cD-ey2O27mGe20ZdXAwHaJ3%26pid%3DApi&f=1",
    "https://duckduckgo.com/i/e3d5d9c8.jpg",
    "https://duckduckgo.com/i/e3d5d9c8.jpg",
    "https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Ftse4.mm.bing.net%2Fth%3Fid%3DOIP.STQgR1_xlV_9fp-U_3UKGgHaFK%26pid%3DApi&f=1",
    "https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Ftse1.mm.bing.net%2Fth%3Fid%3DOIP.eBAy5nnA0bZ04Eit2r01UwHaE8%26pid%3DApi&f=1",
    "https://duckduckgo.com/i/2a33b0cf.jpg",
    "https://duckduckgo.com/i/7e5d69d2.jpg",
    "https://duckduckgo.com/i/538c6d5e.jpg",
    "https://duckduckgo.com/i/3f7407d2.jpg",
    "https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Ftse1.mm.bing.net%2Fth%3Fid%3DOIP.jHVDzAhp_rGcF-eDgng8VQHaDx%26pid%3DApi&f=1",
    "https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Ftse1.mm.bing.net%2Fth%3Fid%3DOIP.HExYzKczlf_BTnA2zxxuPwHaE8%26pid%3DApi&f=1",
    "https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Ftse1.mm.bing.net%2Fth%3Fid%3DOIP._W58c9dzTfe_Z4iPb2QOVQHaH0%26pid%3DApi%26h%3D160&f=1",
    "https://duckduckgo.com/i/cad3558d.jpg",
    "https://duckduckgo.com/i/17190eee.jpg",
    "https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Ftse1.mm.bing.net%2Fth%3Fid%3DOIP.Tpa5Zbu7FSnIwdFhBV1EqAHaE8%26pid%3DApi&f=1",
    "https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Ftse1.mm.bing.net%2Fth%3Fid%3DOIP.x1E7QurwbjXj9BgZU2UspgHaFj%26pid%3DApi&f=1",
    "https://duckduckgo.com/i/a178ce84.jpg",
    "https://duckduckgo.com/i/14c232b2.jpg",
    "https://duckduckgo.com/i/b14202f5.jpg",
    "https://duckduckgo.com/i/0566f8ac.jpg",
    "https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Ftse1.mm.bing.net%2Fth%3Fid%3DOIP.sBVNk4ON-cKKMWHakmd4OgHaEL%26pid%3DApi&f=1",
    "https://duckduckgo.com/i/2f1e0dd4.jpg",
    "https://duckduckgo.com/i/1298dc64.jpg",
    "https://duckduckgo.com/i/e02c4333.jpg",
    "https://duckduckgo.com/i/c26f82da.jpg",
    "https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Ftse1.mm.bing.net%2Fth%3Fid%3DOIP.AHuFROj8MUa53GyTOmfyAQHaEG%26pid%3DApi&f=1",
    "https://duckduckgo.com/i/7792eee4.jpg",
    "https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Ftse1.mm.bing.net%2Fth%3Fid%3DOIP.YRjnn984Nss4Q6YdjrdWKQHaHX%26pid%3DApi&f=1",
    "https://duckduckgo.com/i/0bf631f8.jpg",
]

In [10]:
image_dict = {plant_names[i] : url_list[i] for i in range(len(plant_names))}

In [12]:
for key in image_dict:
    cleaned_dict[key]["Image_Url"] = image_dict[key]

In [14]:
    
fp = open("ui_plant_scrape.json", "w")
json.dump(cleaned_dict, fp)
fp.close()

In [1]:
# get texts from website
# 8 - 12 - 2020
import requests
from bs4 import BeautifulSoup
import shutil
import re
import time
import json
import unicodedata
import html
import re


In [2]:
def clean_text(text):
    text = text.replace("\r\n", " ")
    text = text.replace("\n", " ")
    text = text.replace("\r", " ")
    text = text.replace("\t", " ")
    text = text.replace(u'\xa0', u' ')
    text.strip()
    return text

main_url = "http://www.gardening.cornell.edu/homegardening/sceneb771.html"
base_url = "http://www.gardening.cornell.edu/homegardening/"
page = requests.get(main_url)
soup = BeautifulSoup(page.content, 'html.parser')
html_pages = []
for link in soup.find_all('a'):
    html_pages.append(link.get('href'))
formatted_pages = []
for page in html_pages:
    if page != None and "scene" in str(page):
        formatted = base_url + page
        formatted_pages.append(formatted)
        
final_dict = {}
for url in formatted_pages:
    
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    page_text = soup.get_text("|", strip=True)

    bolds = soup.find_all(['b'])
    name = bolds[1].text.strip()
    
    final_dict[name] = page_text
    
print(final_dict)

{'Tomatoes': "Explore Cornell - Home Gardening - Vegetable Growing Guides - Growing Guide|Search Cornell|Home Gardening|Vegetable Growing Guides|Growing Guide|[+]|larger image|Tomatoes|Vegetable (Warm Season) - Tomatoes, Peppers, Eggplant|Lycopersicon esculentum|Solanaceae Family|The most popular garden vegetable crop, tomatoes come in a wide range\r\nof sizes, shapes and colors. Choose determinate varieties for early\r\nharvest or cool conditions. Compact varieties are also good choices for\r\ncontainers and planting in flower beds.|Site and Plant Characteristics|Growing Information|Varieties|Site Characteristics|Sunlight:|full sun|Tomatoes need at least 8 hours of direct sun daily.|Soil conditions:|tolerates acid soil|requires well-drained soil|Prefers well-drained, fertile soil, high in organic matter. Clays and\r\nloams produce the highest yields. But lighter soils that drain and warm\r\nquickly can produce earlier harvests -- particularly if they are on a\r\nslight slope to the so

In [3]:
no_spacing_dict = {}
for key in final_dict:
    cleaned_of_spaces = final_dict[key].replace("\t", "").replace("\n", "").replace("\r", "")
    no_spacing_dict[key] = cleaned_of_spaces
print(no_spacing_dict)

{'Tomatoes': "Explore Cornell - Home Gardening - Vegetable Growing Guides - Growing Guide|Search Cornell|Home Gardening|Vegetable Growing Guides|Growing Guide|[+]|larger image|Tomatoes|Vegetable (Warm Season) - Tomatoes, Peppers, Eggplant|Lycopersicon esculentum|Solanaceae Family|The most popular garden vegetable crop, tomatoes come in a wide rangeof sizes, shapes and colors. Choose determinate varieties for earlyharvest or cool conditions. Compact varieties are also good choices forcontainers and planting in flower beds.|Site and Plant Characteristics|Growing Information|Varieties|Site Characteristics|Sunlight:|full sun|Tomatoes need at least 8 hours of direct sun daily.|Soil conditions:|tolerates acid soil|requires well-drained soil|Prefers well-drained, fertile soil, high in organic matter. Clays andloams produce the highest yields. But lighter soils that drain and warmquickly can produce earlier harvests -- particularly if they are on aslight slope to the south or southeast. Can to

In [9]:
clean_words_dict = {}
for key in no_spacing_dict:
    cleaned_of_words = no_spacing_dict[key].replace("|[+]|", "|").replace("|larger image|", "|").replace("Explore Cornell - Home Gardening - Vegetable Growing Guides - Growing Guide", "|").replace("Search Cornell","|").replace("Home Gardening", "|").replace("Vegetable Growing Guides", "|").replace("Growing Guide", "|")
    split_list = cleaned_of_words.split("|")
    split_list = list(filter(lambda s : s != '', split_list))
    clean_words_dict[key] = split_list
print(clean_words_dict)

{'Tomatoes': ['Tomatoes', 'Vegetable (Warm Season) - Tomatoes, Peppers, Eggplant', 'Lycopersicon esculentum', 'Solanaceae Family', 'The most popular garden vegetable crop, tomatoes come in a wide rangeof sizes, shapes and colors. Choose determinate varieties for earlyharvest or cool conditions. Compact varieties are also good choices forcontainers and planting in flower beds.', 'Site and Plant Characteristics', 'Growing Information', 'Varieties', 'Site Characteristics', 'Sunlight:', 'full sun', 'Tomatoes need at least 8 hours of direct sun daily.', 'Soil conditions:', 'tolerates acid soil', 'requires well-drained soil', 'Prefers well-drained, fertile soil, high in organic matter. Clays andloams produce the highest yields. But lighter soils that drain and warmquickly can produce earlier harvests -- particularly if they are on aslight slope to the south or southeast. Can tolerate slightly acidsoils, as low as pH 5.5. But produces best when pH is 6.0 to 6.8.Consistent moisture needed to p

In [10]:
fp = open("ui_plant_scrape.json", "r")
cleaned_dict = json.load(fp)
fp.close()

for key in cleaned_dict:
    cleaned_dict[key]["Full Summary"] = clean_words_dict[key]
    
print(cleaned_dict)

{'Tomatoes': {'Sunlight Conditions': 'full sun', 'Plant Lifecycle': 'annual', 'Plant Care Difficulty': 'moderately difficult', 'Height': '2 to 6 feet', 'Spread': '2 to 6 feet', 'Optimal Germination Temperature': '60 F to 95 F       - Germinates best at 75 F to 90 F.  Germinates very slowly at cooler temperatures.', 'Time to Germination': 'Days to emergence: 6 to 12       - About 1 week at 75 F.', 'Soil Depth': '3 Inches', 'Full Summary': ['Tomatoes', 'Vegetable (Warm Season) - Tomatoes, Peppers, Eggplant', 'Lycopersicon esculentum', 'Solanaceae Family', 'The most popular garden vegetable crop, tomatoes come in a wide rangeof sizes, shapes and colors. Choose determinate varieties for earlyharvest or cool conditions. Compact varieties are also good choices forcontainers and planting in flower beds.', 'Site and Plant Characteristics', 'Growing Information', 'Varieties', 'Site Characteristics', 'Sunlight:', 'full sun', 'Tomatoes need at least 8 hours of direct sun daily.', 'Soil conditions

In [11]:
    
fp = open("ui_plant_scrape.json", "w")
json.dump(cleaned_dict, fp)
fp.close()